Rabin Karp algoritması bir pattern eşleştirme algoritmasıdır. Biz bu örnekte bir yazının içinde sözcük araması yapacağız. Ancak iki boyutlu bir sinyalde belli bir örüntü aramak gibi daha kompleks işlemlerde de kullanmak mümkündür. Bunu yapmak için , verilen küme (bu örnkete yazı) ve verilen örüntü (bu örnekte aradığımız küçük sözcük) için hash değeri hesaplayacağız. Hash değeri verilen parçaları matematiksel olarak ifade etmemizi sağlar. hash değerleri kıyaslamak tek tek harf kıyaslamaktan daha verimlidir. 

text = AAAAAAAAAH  9 A ve 1 H  
pattern = AAAAAAAAH  8A ve bir H  

eğer direkt olarak ilk sırada harf karşılaştırmaya başlarsak 8 tane a eşleştikten sonra A ve H kıyaslayıp eşit olmadığını görecek bir sonraki sıraya geçip kıayslama yapacak ve 9 karşılaştırmadan sonra eşleşmeyi bulacak.

Eğer hash hesabı yaparsak her şey daha rahat olur. 

Peki hash hesabı nedir?


## karaktere bağlı değerler üretmek 

Her karakteri belirten sayısal bir değer vardır 

In [2]:
print(ord('A')) 

65


In [3]:
print(ord('a'))

97


In [4]:
print(ord('1')) 

49


stringi oluşturan karakterlerin değerini toplamak basit bir hesap çözümü

In [5]:
def simple_hash(string):

    hash_value = 0
    for char in string:
        hash_value += ord(char)  #it returns an integer representing the Unicode code of the character passed to it.
    return hash_value

In [6]:
input_string = "hello"
hash_value = simple_hash(input_string)
print("Hash value of '{}': {}".format(input_string, hash_value))

Hash value of 'hello': 532


peki ama bu bize niye yarasın ki? Rolling hash diye bişi yapcaz

ascii de 256 muhtemel değer var bunlar büyük küçük harfler ve semboller için 0dan 255e değişen değerlerdir .

bizde arama yaptığımız metin (text) içinde aradığımız örüntümüz(pattern) ile aynı boyutta parçalar alıp hashlarına bakacağız. Eğer hashlar aynı değerdeyse oturup tek tek karakterleri karşılaştıuracağız.


hashları karşılaştırmak tek tek karakter karşılaştırmaktan çok daha kolaydır. Bu yüzdenbn gereksiz harf karşılaştırmaktan kaçınmak çok önemlidir

Gereksiz harf karşılaştırma miktarını en aza indirmek için en iyi yöntem hash değerlerinin olabildiğince farklı olmasından emin olmaktır.

örneğin 

abca texti içinde 

abc ve bca patternleri,  eğer sadece karkter değerlerini toplarsak aynı hashı üretecek.bu iki kere kontrol demek.

bunun yerine  eğer pozisyondan da etkilenen bir hash üretimi yaparsak farklı değerler alınmasını sağlarız.

Bu mantık aslında rolling hash ve check sum olarakta bilinir. Hatta check sum emthodu interntten yaptığınız büyük indirmelerin ( mesela bir linux distrosu) doğruluğunu kontrol eder. Eğer hatalı bir indirme yaptıysan (indirme sırasında bozuk bir dosya inmesi paket kaybı olabilir düşük bir ihtimal olsada cihazını riske atacak bir indirmede, işletim sistemi güncellemesi gibi, kontrol etmek iyidir.)

## posziyona bağlı hash değeri

basamağa bağlı hash değeri üreten bir sistem yapalım

aşağıda arama yaptığımız patternin basamak sayısına bağlı farklı değerler oluşturan bir sistem yapacağız

In [7]:
pattern = "abcadacac"
q = 17 # bir asal sayı ile hash değerlernin farklılığını arttıracağız 
d = 29  # mümkün olan eleman sayısı, acii karakterl için 256 ama biz küçük harf bakcaz şuan
h=1  # h için bir başlangıc değeri, çarpmada etkisiz eleman 1 seçilir 
for i in range(len(pattern)-1):
		h = (h*d) % q
		print(h)

12
8
11
13
3
2
7
16


Yukarda gördüğün sistem 1 basamak için 12 2 basmak için 8 üç basamak için 11 değerini üretti ve bizim final değerimiz 16 oldu mevcut uzunluk için 

asal sayı olarak 17 seçmiştik peki ya 13 seçseydik?

In [8]:
pattern = "abcadacac"
q = 13 # bir asal sayı ile hash değerlernin farklılığını arttıracağız 
d = 29  # mümkün olan eleman sayısı, acii karakterl için 256 ama biz küçük harf bakcaz şuan
h=1  # h için bir başlangıc değeri, çarpmada etkisiz eleman 1 seçilir 
for i in range(len(pattern)):
		h = (h*d) % q
		print(h)

3
9
1
3
9
1
3
9
1


gördüğün gibi hash kendini tekrar etti ancak 17 için özel sayıalar oluştu. asal sayıyı duruma göre seçmek biraz deneyim ister ancak büyük asal sayılar daha başarılı olur
farklı basamak uzunlukları için aynı hash değerini veriyor. bu bizim 1 ve 4 basamaklı pattern farkının hasha bakarak anlamamızı engeller çünkü ikiside 9 değerinde

biz her basamak için farklı hashler oluşturmayı tercih ediyoruz

## karakter ascii değeri ile posizyon değerinin birleştirilmesi 

bu aşamadan sonra işleri kolaylaştırmak adına daha kısa bir pattern kullanacağız. ama unutma bu sistemin verimliliği pattern büyüdükçe kendini belli edecek

şuan 3 karakterlik bir pattern (aradığımız yazı substring) için hash hesaplayacığız ve ardından uzun bir text (arama yaptığımız büyük metin string ) girdisinin patternimizle eşleşen uzunluktaki kısmı için hash değeri arayacağız. 

Sonuçta 3 karakterli bir pattern, textin 4 karakterlik bir parçasının hash değeri ile eşleşse de biz  pattern ve seçilen 4 karakterli alanın aynı olmadığını biliyoruz. Bu yüzden pattern uzunluğumuz ile eşit miktardaki parçalara bakarak eşleştirme yapacaığız. 


In [9]:
pattern = "abca"
q = 13 # bir asal sayı ile hash değerlernin farklılığını arttıracağız 
d = 29  # mümkün olan eleman sayısı, acii karakterl için 256 ama biz küçük harf bakcaz şuan
h=1  # h için bir başlangıc değeri, çarpmada etkisiz eleman 1 seçilir 
p=0  # p pattern için hash değeri. toplama ile belirleyeceğimiz için toplamanın etkisiz elamnını kullandık

for i in range(len(pattern)):
		print(f"karakter: {pattern[i]}")
		h = (h*d) % q
		print( f"pozisyona bağlı değer: {h}")
		print( f"karakterin ascii ifadesine bağlı değer: {ord(pattern[i])}")
		p = (d*p + ord(pattern[i])) % q
		print( f"pozisyona ve karakterin ascii ifadesine bağlı değer: {p}")

		print("")
		print("************")
		print("")

karakter: a
pozisyona bağlı değer: 3
karakterin ascii ifadesine bağlı değer: 97
pozisyona ve karakterin ascii ifadesine bağlı değer: 6

************

karakter: b
pozisyona bağlı değer: 9
karakterin ascii ifadesine bağlı değer: 98
pozisyona ve karakterin ascii ifadesine bağlı değer: 12

************

karakter: c
pozisyona bağlı değer: 1
karakterin ascii ifadesine bağlı değer: 99
pozisyona ve karakterin ascii ifadesine bağlı değer: 5

************

karakter: a
pozisyona bağlı değer: 3
karakterin ascii ifadesine bağlı değer: 97
pozisyona ve karakterin ascii ifadesine bağlı değer: 8

************



Burada h ve p değerlerinin her for döngüsü için kendisi üzerinden hesaplandığını farketmiş olmamız lazım. aşağıdaki örnekte abca yerine fzea için hash hesapladık. Sondaki a için poziyona ve ascii değerine bağlı olan değişirken pozisyona bağlı olan sabit kaldı çünkü ikisde 3. indexte.

In [10]:
pattern = "fzea"
q = 13 # bir asal sayı ile hash değerlernin farklılığını arttıracağız 
d = 29  # mümkün olan eleman sayısı, acii karakterl için 256 ama biz küçük harf bakcaz şuan
h=1  # h için bir başlangıc değeri, çarpmada etkisiz eleman 1 seçilir 
p=0  # p pattern için hash değeri. toplama ile belirleyeceğimiz için toplamanın etkisiz elamnını kullandık

for i in range(len(pattern)):
		print(f"karakter: {pattern[i]}")
		h = (h*d) % q
		print( f"pozisyona bağlı değer: {h}")
		print( f"karakterin ascii ifadesine bağlı değer: {ord(pattern[i])}")
		p = (d*p + ord(pattern[i])) % q
		print( f"pozisyona ve karakterin ascii ifadesine bağlı değer: {p}")

		print("")
		print("************")
		print("")

karakter: f
pozisyona bağlı değer: 3
karakterin ascii ifadesine bağlı değer: 102
pozisyona ve karakterin ascii ifadesine bağlı değer: 11

************

karakter: z
pozisyona bağlı değer: 9
karakterin ascii ifadesine bağlı değer: 122
pozisyona ve karakterin ascii ifadesine bağlı değer: 12

************

karakter: e
pozisyona bağlı değer: 1
karakterin ascii ifadesine bağlı değer: 101
pozisyona ve karakterin ascii ifadesine bağlı değer: 7

************

karakter: a
pozisyona bağlı değer: 3
karakterin ascii ifadesine bağlı değer: 97
pozisyona ve karakterin ascii ifadesine bağlı değer: 1

************



## Hash değerleri eşleşince harf karşılaştırma 

hash değerleri eşleşince artık arakterlere bakma sırası gelir 

In [34]:
i = 2 # bu i hashleri kontrol ettiğimiz for döngğsğne ait olacak
p,t =10,10 # bunların hash değerleri olduğunu düşün
pat ="abc"
txt="aaabc"
M = len(pat)
matchCounter = 0

# burayı bir for döngüsünün içi olarak düşün 
# sırayla text içindeki hashleri tek tek bakıyor 
# hash değerleri eşit ise  bu if içine girecek
if p == t:                    
    for j in range(M):  # pattern uzunluğunda bir döngü yapılacak
        if txt[i+j] != pat[j]:  # i dış döngünün değeri olacktı i text içindeki pozisyonumuz  
            matchCounter=0   # eğer eşleşme yoksa  eşleşme sayacı sıfırlanır 
            break
        else:                   # eğer değerler aynı isse eşleşme sayacı artar 
            matchCounter += 1
    
    if matchCounter == M:        # eğer pattern uzunluğunda eşleşme olursa başarılı olunmuştur
	    print("Pattern found at index " + str(i))

Pattern found at index 2


##  Arama yapılacak text üzerinde hareket etmek 

3 karakterlik bir pattern için bir hash oluşturduk ve bunu textin sıfırnıcı indeksinden başlayıp ilk üç karakteri ile kıyasladık ancak eşleşmedi şimdi birinci indexten başlayarak 3 elemanı kontrol etmemiz lazım 

örnek ile bakalım
text= "abcde"
pattern = "cde"

abc için hash hesabı yaptık ve patternin  hashı ile eşleşmedi.
bcd ye geçmemiz lazım 

ancak b ve c için hem poziyona bağlı hemde harf değerine bağlı hesabı abc için yaptık. bcd için hesap yaparken b ve c yi tekrar hesaplama gereği yok 


x leri boş yer tutucu olarak düşünelim 


kıyaslama bir:   
a b c d e     
c d e x x       



kıyaslama iki:     
a b c d e     
x c d e x     



kıyaslama üç:
a b c d e      
x x c d e        


5 karakterli bir text üzerinde 3 karakterli bir patterni boydan boya artmak için 3 kıyaslama gerekti 

yapılacak kıyaslama sayısını hesaplamak için len(text)-len(pattern)+1 diyebiliriz 

-- kısa bir hatırlatma   

elimizde uzun bir yazı var(text)   
bu yazı içinde X sayıda karakter içeren bir yazı arıyoruz (pattern)   

uzun yazının ilk X sayıda karakteri ve patternimi için hash değerleri hesaplayacağız  

eğer değerler eşit ise harflerin eşitliğine bakacağız  

eğer hash değerleri farklı ise uzun yazı içinde bir karakter yana kayacağız   


peki yana kayınca hash hesabı ne olacak 

hash hesabımız asında her karakterin konumuna ve ascii karşılığına bağlı değerler toplamı, Z sayıda karakter içeren bir pattern arar iken index 0 dan index Z-1 uzunluğunda bir hash hesabı ile başlanacak sonra index 1den z sonra 2den z+1 olarak gidecek.

her bir yan harekette hash değerine yeni giren karakteri eklerken eskisini çıkartıyoruz

öncelikle  başlangıç değerlerini hseaplamak lazım. Text üzerinde pattern uzunlıuğunda bir alana bakacağız. Bu alana window denilecek. 

In [42]:
# text and pattern
text="pattern"
pattern="pattern"


# d is the number of characters in the input alphabet
d = 256
# q -> A prime number
q=101

#initial variables to calclate hash
p = 0 # hash value for pattern
t = 0 # hash value for txt
h = 1

# initial hash values 
M = len(pattern) #need to find the pattern length to create window

for i in range(M):
		p = (d*p + ord(pattern[i])) % q
		t = (d*t + ord(text[i])) % q
		
print(f"value for pattern hash : {p}   and value for windown hash: {t}")

value for pattern hash : 18   and value for windown hash: 18


değerler eşlemiyor demekki window ve pattern eşlleşmedi o zaman windowu kaydırmak lazım.
bu kaydırma için text uzunluğunu bilmek lazım ki eğer sona gelip bulamadıysak bulamdığımızı fark edelim


In [47]:
N = len(text)
j=0

for i in range(M-1):
    h = (h*d) % q

for i in range(M):
    p = (d*p + ord(pattern[i])) % q
    t = (d*t + ord(text[i])) % q

for i in range(N-M+1):
    print(f" {p}  {t}")
    if p == t:
        for b in range(M):
            if text[i+j] != pattern[j]:
                break
            else:
                j += 1 
        if j == M:
            print("Pattern found at index " + str(i))
   
    if i < N-M:
        t = (d*(t-ord(text[i])*h) + ord(text[i+M])) % q

        # We might get negative values of t, converting it to
        # positive
        if t < 0:
            t = t+q


 92  92
 1  7
 2  7
 3  7
 4  7
 5  7
 6  7
 7  7
Pattern found at index 0


In [30]:

d = 256

def search(pat, txt, q):
	M = len(pat)
	N = len(txt)
	i = 0
	j = 0
	p = 0 # hash value for pattern
	t = 0 # hash value for txt
	h = 1

	# The value of h would be "pow(d, M-1)%q"
	for i in range(M-1):
		h = (h*d) % q


	for i in range(M):
	
		p = (d*p + ord(pat[i])) % q
		
		t = (d*t + ord(txt[i])) % q
	
		

	
	for i in range(N-M+1):
		
	
		if p == t:
			
			for b in range(M):
				if txt[i+j] != pat[j]:
					break
				else:
					j += 1

			
			if j == M:
				print("Pattern found at index " + str(i))

		
		
		if i < N-M:
			t = (d*(t-ord(txt[i])*h) + ord(txt[i+M])) % q

			# We might get negative values of t, converting it to
			# positive
			if t < 0:
				t = t+q


# Driver Code
if __name__ == '__main__':
	txt = "great pattern here "
	pat = "pattern"

	# A prime number
	q = 101

	# Function Call
	search(pat, txt, q)


Pattern found at index 6
